In [6]:
import torch
from models.transfer_CNN import ResNet

import utilities.interpt_utils as iu
import utilities.model_utils as mu

import numpy as np
from sklearn.metrics import mean_squared_error
import json, os
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
config_tuple = (3,2,1)
datasize_cut = 1000
X_gcm,y_gcm = mu.prepare_data(config_tuple, 
                          param_names=['csd', 'C_DD', 'C_DG'], 
                          all_batches=False,
                          batches=[33], 
                          datasize_cut=datasize_cut,
                          maxwell=True)

X_lcm,y_lcm = mu.prepare_data(config_tuple, 
                          param_names=['csd', 'C_DD', 'C_DG'], 
                          all_batches=False,
                          batches=[33], 
                          datasize_cut=datasize_cut,
                          maxwell=False)

Loading data from ./ALICE/N-2_S-1\96x96\batch-33\datapoints.h5
Preprocessing data...
Data preprocessed. Using 1000 datapoints.
Loading data from ./ALICE/N-2_S-1\96x96\batch-33\datapoints.h5
Preprocessing data...
Data preprocessed. Using 1000 datapoints.


In [8]:
print(f"LCM:\n c_dd = {mu.reconstruct_capacitance_matrices(config_tuple, y_lcm[0])[0]}\n c_dg = {np.array(mu.reconstruct_capacitance_matrices(config_tuple, y_lcm[0])[1])}")
print(f"GCM:\n c_dd = {mu.reconstruct_capacitance_matrices(config_tuple, y_gcm[0])[0]}\n c_dg = {np.array(mu.reconstruct_capacitance_matrices(config_tuple, y_gcm[0])[1])}")

LCM:
 c_dd = [[  0.           2.10938311   0.92016703]
 [  2.10938311   0.           5.16392803]
 [  0.92016703   5.16392803 100.        ]]
 c_dg = [[9.7009726e+00 9.5628999e-02 0.0000000e+00]
 [4.1499999e-03 1.1763789e+01 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.6455448e+01]]
GCM:
 c_dd = [[ 12.82615185  -2.10938311  -0.92016703]
 [ -2.10938311  19.04125023  -5.16392803]
 [ -0.92016703  -5.16392803 122.53954315]]
 c_dg = [[9.7009726e+00 9.5628999e-02 0.0000000e+00]
 [4.1499999e-03 1.1763789e+01 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.6455448e+01]]


In [ ]:
def load_model(model_name: str, mc_type: str, weights_path: str):
    with open(os.path.join(os.path.dirname(weights_path), 'config.json'), 'r') as f:
        config = json.load(f)
    
    if model_name == ''
    model = ResNet(config_tuple=config_tuple, 
                  name=model_name,
                  base_model='resnet18',
                  pretrained=False,
                  custom_head=[4096,512], 
                  dropout=0.1)

In [31]:
LCM_weights_path_10 = './Results_FINAL/Results/resnet10/Rn10-2-1_20250517_182042/Rn10-2-1-.pth'
GCM_weights_path_10 = './Results_FINAL/Results/resnet10/Rn10-2-1_20250517_172057/Rn10-2-1-.pth'

LCM_weights_path_18 = './Results_FINAL/Results/resnet18/Rn18-2-1_20250517_180722/Rn18-2-1-.pth'
GCM_weights_path_18 = './Results_FINAL/Results/resnet18/Rn18-2-1_20250517_174156/Rn18-2-1-.pth'

rn18_lcm = ResNet(config_tuple=config_tuple, 
                  name="LCM-21-Rn18-Interp",
                  base_model='resnet18',
                  pretrained=False,
                  custom_head=[4096,512], 
                  dropout=0.1)
rn18_lcm = mu.load_model_weights(rn18_lcm, LCM_weights_path_18)

rn18_gcm = ResNet(config_tuple=config_tuple, 
                  name="GCM-21-Rn18-Interp",
                  base_model='resnet18',
                  pretrained=False,
                  custom_head=[4096,512], 
                  dropout=0.1)
rn18_gcm = mu.load_model_weights(rn18_gcm, GCM_weights_path_18)

print(f"Loaded models results from traning:")
LCM_res = pd.read_csv(os.path.join(os.path.dirname(LCM_weights_path_18), 'results.csv'))
GCM_res = pd.read_csv(os.path.join(os.path.dirname(GCM_weights_path_18), 'results.csv'))
print(f"LCM:\n{LCM_res}\n")
print(f"GCM:\n{GCM_res}")

Loaded models results from traning:
LCM:
                 Metric      Value
0             Test Loss   0.416754
1  Test Accuracy Global   0.000000
2   Test Accuracy Local  65.490000
3                   MSE   0.416754
4                   MAE   0.255666
5                    R2   0.194831

GCM:
                 Metric      Value
0             Test Loss   0.663633
1  Test Accuracy Global   0.000000
2   Test Accuracy Local  48.970000
3                   MSE   0.663633
4                   MAE   0.361315
5                    R2   0.377166


In [32]:
rn18_lcm.eval()
rn18_gcm.eval()

with torch.no_grad():
    pred_lcm = rn18_lcm(X_lcm)
    pred_gcm = rn18_gcm(X_gcm) 

In [33]:
print("Obtained results on unsampled data:")
MSE_lcm = mean_squared_error(pred_lcm, y_lcm)
MSE_gcm = mean_squared_error(pred_gcm, y_gcm)

print(f"MSE_lcm: {MSE_lcm}, MSE_gcm: {MSE_gcm}")

l2_norms_lcm = np.linalg.norm(y_lcm - pred_lcm, axis=1)
l2_norms_gcm = np.linalg.norm(y_gcm - pred_gcm, axis=1)
print(f"L2 norms LCM (mean): {np.mean(l2_norms_lcm)}, L2 norms GCM (mean): {np.mean(l2_norms_gcm)}")

Obtained results on unsampled data:
MSE_lcm: 0.42892152070999146, MSE_gcm: 0.7223401665687561
L2 norms LCM (mean): 2.2539143562316895, L2 norms GCM (mean): 2.823944330215454


In [39]:
# Create a new array with the l2 norms, y_lcm, pred_lcm, and X_lcm
interp_lcm, interp_gcm = [], []
for i in range(l2_norms_lcm.shape[0]):
    interp_lcm.append([l2_norms_lcm[i], y_lcm[i], pred_lcm[i], X_lcm[i]])
    interp_gcm.append([l2_norms_gcm[i], y_gcm[i], pred_gcm[i], X_gcm[i]])

interp_lcm = sorted(interp_lcm, key = lambda x: x[0])
interp_gcm = sorted(interp_gcm, key = lambda x: x[0])

In [ ]:
def get_saliency_maps(X, interpretor_model_lcm, interpretor_model_gcm, save_images: bool = True, output_path: str = './Results_FINAL/Results/Interp_res/All_methods/Rn18'):
    saliency_maps_gcm = [] 
    for i, x in enumerate(X):
    
        dict_masks_gcm = {
                      'Grad-CAM++':  interpretor_model_gcm.compute_gradcam_plus_plus(x), 
                      'Guided Grad-CAM': interpretor_model_gcm.compute_guided_gradcam(x),
                      'ScoreCAM': interpretor_model_gcm.compute_scorecam(x),
                      'SmoothGrad': interpretor_model_gcm.compute_smoothgrad(x)
                      }
        if save_images:
            interpretor_model_gcm.visualize_all_masks(x,
                                                      dict_masks_gcm,
                                                      save_path=f'{output_path}/GCM/gradcam_gcm_{i}.png',
                                                      show=False)
        
        saliency_maps_gcm.append({'X'        


In [ ]:
interpretor_model_lcm = iu.InterpModel(rn10_lcm)
interpretor_model_gcm = iu.InterpModel(rn10_gcm)
X = [interp_lcm[i][-1] for i in range(len(X_gcm))][:20]
for i, x in enumerate(X):
    dict_masks_gcm = {'Grad-CAM++':  interpretor_model_gcm.compute_gradcam_plus_plus(x), 
                      'Guided Grad-CAM': interpretor_model_gcm.compute_guided_gradcam(x),
                      'ScoreCAM': interpretor_model_gcm.compute_scorecam(x),
                      'SmoothGrad': interpretor_model_gcm.compute_smoothgrad(x)
                      }
    
    interpretor_model_gcm.visualize_all_masks(x, 
                                              dict_masks_gcm, 
                                              save_path=f'./Results_FINAL/Results/Interp_res/All_methods/Rn18/GCM/gradcam_gcm_{i}.png', 
                                              show=False)

    interp_gcm[i].append(dict_masks_gcm)

    dict_masks_lcm = {'Grad-CAM++':  interpretor_model_lcm.compute_gradcam_plus_plus(x), 
                      'Guided Grad-CAM': interpretor_model_lcm.compute_guided_gradcam(x),
                      'ScoreCAM': interpretor_model_lcm.compute_scorecam(x),
                      'SmoothGrad': interpretor_model_lcm.compute_smoothgrad(x)
                      }
    interpretor_model_lcm.visualize_all_masks(x, 
                                              dict_masks_lcm, 
                                              save_path=f'./Results_FINAL/Results/Interp_res/All_methods/Rn18/LCM/gradcam_lcm_{i}.png', 
                                              show=False)
    interp_lcm[i].append(dict_masks_lcm)


c:\Users\barte\anaconda3\envs\qdarts\lib\site-packages\torch\nn\modules\module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
d:\!!!_PDF\Fizyka i matma studia\!!!_Leiden\!! projekty 2024_25\Evert van Niewenburg\potencial projects\Qdarts projects\qdarts_code\learning_parameters\utilities\interpt_utils.py:952: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [71]:
interp_lcm_dict_1 = {'L2':  [x for x in l2_norms_lcm],
                     'y' : [x for x in y_lcm],
                     'y_pred' : [x for x in pred_lcm],
                     'X' : [x for x in X_lcm]}

interp_lcm_dict_2 = {k:[d[k] for d in [x[4] for x in interp_lcm]] for k in [x[4] for x in interp_lcm][0] }


interp_lcm_dict = interp_lcm_dict_1 | interp_lcm_dict_2
np.savez_compressed("./Interp_results/interp_lcm.npz", **interp_lcm_dict)  


interp_gcm_dict_1 = {'L2':  [x for x in l2_norms_gcm],
                     'y' : [x for x in y_gcm],
                     'y_pred' : [x for x in pred_gcm],
                     'X' : [x for x in X_gcm]}

interp_gcm_dict_2 = {k:[d[k] for d in [x[4] for x in interp_gcm]] for k in [x[4] for x in interp_gcm][0] }


interp_gcm_dict = interp_gcm_dict_1 | interp_gcm_dict_2
np.savez_compressed("./Interp_results/interp_gcm.npz", **interp_gcm_dict)    

In [72]:
loaded_gcm = np.load("./Interp_results/interp_gcm.npz")


In [1]:
%matplotlib inline
interpretor_model_lcm = iu.InterpModel(rn18_lcm)
interpretor_model_gcm = iu.InterpModel(rn18_gcm)

# interp_x = [interp_lcm[0][-1], interp_gcm[250][-1], interp_gcm[500][-1],interp_gcm[750][-1],interp_gcm[-1][-1]]
# interp_x = [ x[-1] for x in interp_lcm[0:10]]
interp_x = [interp_lcm[1][-1], interp_gcm[16][-1], interp_gcm[118][-1],interp_gcm[128][-1],interp_gcm[471][-1], interp_gcm[692][-1]]

gradcam_map_gcm = interpretor_model_gcm.compute_gradcam(interp_x)
interpretor_model_gcm.visualize_gradcam(interp_x, gradcam_map_gcm, save_path='./Interp_results/Figs_GradCam/GCM/all_gradcam_maps_gcm.png')


gradcam_map_lcm = interpretor_model_lcm.compute_gradcam(interp_x)
interpretor_model_lcm.visualize_gradcam(interp_x, gradcam_map_lcm, save_path='./Interp_results/Figs_GradCam/LCM/all_gradcam_maps_lcm.png')

NameError: name 'iu' is not defined

In [41]:
import utilities.plots_utils as plu
seeds = [42,5,997]
main_folder_paths = ['.\Results\Results-GCM-NL', 
                         '.\Results\Results-LCM-NL',
                         '.\Results\Results-GCM-NO', 
                         '.\Results\Results-LCM-NO',
                         '.\Results\Results-GCM-NO-5-3-2-Rn10-Rn18', 
                         '.\Results\Results-LCM-NO-5-3-2-Rn10-Rn18']
    
devices = ['2-0', '2-1', '2-1-n8e5', '2-1-n2e5']
resnet_versions = ['Rn10', 'Rn12', 'Rn14', 'Rn16', 'Rn18']
systems = [f'{rv}-{device}' for rv in resnet_versions for device in devices]
systems.append('Rn10-3-2')
systems.append('Rn18-3-2')

x = plu.plt_mse_chosen_element(main_folder_paths = main_folder_paths,
                               systems = systems,
                               seeds = seeds,
                               element_index = 0,
                               interaction_type = 'dd',
                               output_path = '.\Results\Figs')
    

Model folder resnet12 not found for system Rn12-2-0
Model folder resnet12 not found for system Rn12-2-1
Model folder resnet12 not found for system Rn12-2-1-n8e5
Model folder resnet12 not found for system Rn12-2-1-n2e5
Model folder resnet14 not found for system Rn14-2-0
Model folder resnet14 not found for system Rn14-2-1
Model folder resnet14 not found for system Rn14-2-1-n8e5
Model folder resnet14 not found for system Rn14-2-1-n2e5
Model folder resnet16 not found for system Rn16-2-0
Model folder resnet16 not found for system Rn16-2-1
Model folder resnet16 not found for system Rn16-2-1-n8e5
Model folder resnet16 not found for system Rn16-2-1-n2e5
Model folder resnet12 not found for system Rn12-2-0
Model folder resnet12 not found for system Rn12-2-1
Model folder resnet12 not found for system Rn12-2-1-n8e5
Model folder resnet12 not found for system Rn12-2-1-n2e5
Model folder resnet14 not found for system Rn14-2-0
Model folder resnet14 not found for system Rn14-2-1
Model folder resnet14 no

In [42]:
x

{'GCM': {'2-0': {42: {'Rn10': {'pred': array([[ 1.4055081e+01, -2.0290475e+00,  1.6678215e+01, ...,
              6.7158774e-02,  7.2001338e-02,  1.4586901e+01],
            [ 1.6683512e+01, -1.5105134e+00,  1.2814993e+01, ...,
              2.4969123e-02,  6.9539875e-02,  1.1249265e+01],
            [ 1.8149639e+01, -1.4742512e+00,  1.5246061e+01, ...,
              8.7226436e-02,  2.4347350e-02,  1.3737429e+01],
            ...,
            [ 1.3788210e+01, -1.4699675e+00,  1.3797599e+01, ...,
              1.1365536e-01,  2.1056455e-02,  1.2294861e+01],
            [ 1.4777243e+01, -2.7057712e+00,  1.3228412e+01, ...,
              8.6907715e-02,  5.4364577e-02,  1.0477430e+01],
            [ 1.7908264e+01, -2.0752099e+00,  1.5318080e+01, ...,
              1.9118656e-01,  1.3442494e-02,  1.3224663e+01]], dtype=float32),
     'true': array([[ 1.4156209e+01, -2.0928221e+00,  1.6745831e+01, ...,
              1.1347900e-01,  1.1498300e-01,  1.4538026e+01],
            [ 1.6801332e+01,

In [4]:
import numpy as np
y = np.array(x['LCM']['2-1'][42]['Rn10']['pred'])
y_true = np.array(x['LCM']['2-1'][42]['Rn10']['true'])
y.shape

(3200, 15)

In [16]:
def mse_per_element(y,y_true,element_index):
    return np.sum([(a[element_index]-b[element_index])**2 for a,b in zip(y_true,y)])/len(y_true)        

In [ ]:
element_index = 1
mse_per_element_to_plot = dict(x)

devices = ['2-0', '2-1', '2-1-n8e5', '2-1-n2e5', '3-2']
architectures = ['Rn10', 'Rn12', 'Rn14', 'Rn16', 'Rn18']
seeds = [42,5,997]
for model_type in ['GCM', 'LCM']:
    for device in devices:
        for seed in seeds:
            for arch in architectures:
                try:
                    x[model_type][device][seed][arch]['mse'] = mse_per_element(x[model_type][device][seed][arch]['pred'],
                                                                                                     x[model_type][device][seed][arch]['true'],
                                                                                                     element_index)
                    x[model_type][device][seed][arch].pop('pred')
                    x[model_type][device][seed][arch].pop('true')                                                                                             
                except TypeError:
                    x[model_type][device][seed][arch]['mse'] = np.nan
                    print(f"The entry of such {model_type}/{device}/{seed}/{arch} is NaN. Skipping...")
                    continue
                
np.array(mse_per_element_to_plot).shape

The entry of such GCM/3-2/42/Rn12 is NaN. Skipping...
The entry of such GCM/3-2/42/Rn14 is NaN. Skipping...
The entry of such GCM/3-2/42/Rn16 is NaN. Skipping...
The entry of such GCM/3-2/5/Rn12 is NaN. Skipping...
The entry of such GCM/3-2/5/Rn14 is NaN. Skipping...
The entry of such GCM/3-2/5/Rn16 is NaN. Skipping...
The entry of such GCM/3-2/997/Rn12 is NaN. Skipping...
The entry of such GCM/3-2/997/Rn14 is NaN. Skipping...
The entry of such GCM/3-2/997/Rn16 is NaN. Skipping...
The entry of such LCM/3-2/42/Rn12 is NaN. Skipping...
The entry of such LCM/3-2/42/Rn14 is NaN. Skipping...
The entry of such LCM/3-2/42/Rn16 is NaN. Skipping...
The entry of such LCM/3-2/5/Rn12 is NaN. Skipping...
The entry of such LCM/3-2/5/Rn14 is NaN. Skipping...
The entry of such LCM/3-2/5/Rn16 is NaN. Skipping...
The entry of such LCM/3-2/997/Rn12 is NaN. Skipping...
The entry of such LCM/3-2/997/Rn14 is NaN. Skipping...
The entry of such LCM/3-2/997/Rn16 is NaN. Skipping...


()

In [46]:
x

{'GCM': {'2-0': {42: {'Rn10': {'mse': 0.013223035505491344},
    'Rn12': {'mse': 0.007325529220701487},
    'Rn14': {'mse': 0.0017705558184394866},
    'Rn16': {'mse': 0.014283665384878535},
    'Rn18': {'mse': 0.00096011118994198}},
   5: {'Rn10': {'mse': 0.0045400467825558784},
    'Rn12': {'mse': 0.014827935604293518},
    'Rn14': {'mse': 0.004676342945465015},
    'Rn16': {'mse': 0.0018518376544859128},
    'Rn18': {'mse': 0.0028652161783594434}},
   997: {'Rn10': {'mse': 0.005664943117547654},
    'Rn12': {'mse': 0.00215126824810708},
    'Rn14': {'mse': 0.004621605001465166},
    'Rn16': {'mse': 0.0010998555135048305},
    'Rn18': {'mse': 0.002711737694791624}}},
  '2-1': {42: {'Rn10': {'mse': 0.018819530922306503},
    'Rn12': {'mse': 0.02139701394661134},
    'Rn14': {'mse': 0.03608567353797412},
    'Rn16': {'mse': 0.1288297630586449},
    'Rn18': {'mse': 0.01751048775214488}},
   5: {'Rn10': {'mse': 0.024562670926461833},
    'Rn12': {'mse': 0.03691255661014385},
    'Rn14': 

In [35]:
mse_per_element_to_plot == x

True

In [23]:
len(mse_per_element_to_plot['GCM']['3-2'][42])

5

In [ ]:
y['2-0']['Rn10'][0]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [39]:
'True'in'MW-C_DG-False' 

False